# Workflows Observability - Part 1


## Use native instrumentation from LlamaIndex + OpenTelemetry to fine-grain tracing in your code!





In this notebook, we will go through an example of how to use instrumentation natively implemented in `llama-index` (combined with OpenTelemetry) to define costum span and events within your code. Before we get started:


⭐ Don'f forget to star the `llama-index-workflows` [GitHub repo](https://github.com/run-llama/workflows-py)

🦙☁ Register to [LlamaCloud](https://cloud.llamaindex.ai) not to miss out on all our awesome products

If you have feedback, questions, issues, or you just want to follow us not to miss out on any news, please find us on:

[![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white)](https://github.com/run-llama/)
[![Discord](https://img.shields.io/badge/Discord-%235865F2.svg?style=for-the-badge&logo=discord&logoColor=white)](https://discord.com/invite/eN6D2HQ4aX)
[![X](https://img.shields.io/badge/@llama__index-%23000000.svg?style=for-the-badge&logo=X&logoColor=white)](https://x.com/@llama_index)
[![Bluesky](https://img.shields.io/badge/Bluesky-0285FF?style=for-the-badge&logo=Bluesky&logoColor=white)](https://bsky.app/profile/llamaindex.bsky.social)
[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/company/llamaindex/)

## 1. Setting up

Before diving deep into all of this, let's install all the needed dependencies.

In [1]:
! pip install -q llama-index-workflows llama-index-instrumentation llama-index-llms-openai llama-index-observability-otel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


## 2. Experiment with instrumentation

Let's now play around with `llama-index` dispatcher and see how we can make it work.

Let's start by initializing it:

In [17]:
from llama_index_instrumentation import get_dispatcher

dispatcher = get_dispatcher()

Now we can use the `@dispatcher.span` decorator on a function that we defined to emit spans (containers for events) and use `dispatcher.event` to emit an event (we can define custom events by subclassing the `BaseEvent` class):

In [18]:
from llama_index_instrumentation.base import BaseEvent


class ExampleEvent(BaseEvent):
    data: str


class AnotherExampleEvent(BaseEvent):
    print_statement: str


@dispatcher.span
def example_fn(data: str) -> None:
    dispatcher.event(ExampleEvent(data=data))
    s = "This are example string data: " + data
    dispatcher.event(AnotherExampleEvent(print_statement=s))
    print(s)

## 3. Add OpenTelemetry

We can now add OpenTelemetry so that we can export all our span and events as ordered traces.
We will be using the LlamaIndex integration for that, `llama-index-observability-otel`.


We start by defining a custom `SpanExporter` that can write all our traces to a file:

In [1]:
import os
from pathlib import Path

import json

from llama_index.observability.otel import LlamaIndexOpenTelemetry
from opentelemetry.sdk.trace.export import SpanExporter, SpanExportResult
from opentelemetry.sdk.trace import ReadableSpan
from typing import Optional, Callable, Sequence
from os import linesep


class FileSpanExporter(SpanExporter):
    """Implementation of :class:`SpanExporter` that prints spans to the
    console.

    This class can be used for diagnostic purposes. It prints the exported
    spans to the console STDOUT.
    """

    def __init__(
        self,
        service_name: str | None = None,
        file_path: Optional[os.PathLike[str]] = None,
        formatter: Callable[[ReadableSpan], str] = lambda span: json.dumps(
            json.loads(span.to_json())
        )
        + linesep,
    ):
        if not file_path:
            file_path = "traces.json"
        if Path(file_path).exists():
            raise ValueError(f"File {file_path} already exists")
        self.file_path = file_path
        self.formatter = formatter
        self.service_name = service_name

    def export(self, spans: Sequence[ReadableSpan]) -> SpanExportResult:
        print(f"Writing {len(spans)} spans to {self.file_path}")
        if Path(self.file_path).exists():
            mode = "a"
        else:
            mode = "w"
        with open(self.file_path, mode) as out:
            for span in spans:
                out.write(self.formatter(span))
            out.flush()
        return SpanExportResult.SUCCESS

    def force_flush(self, timeout_millis: int = 30000) -> bool:
        return True

It is important to notice that we are defining here a custom span exporter since it is an easier implementation for notebooks, but there are many exporting options detailed by OpenTelemetry in [this page](https://opentelemetry.io/docs/languages/python/exporters/).

Now we can pass that to the instrumentation class as a span exporter

In [5]:
se = FileSpanExporter(file_path="traces_example.json")

instrumentor = LlamaIndexOpenTelemetry(
    span_exporter=se,
    service_name_or_resource="example_service",
)

And we can try and see how events are registered simply by calling `example_fn` as defined before :)

In [6]:
instrumentor.start_registering()

example_fn(data="Hello world!")

This are example string data: Hello world!


In [7]:
with open("traces_example.json") as f:
    lines = f.readlines()
    print(json.dumps(json.loads(lines[0]), indent=4))

{
    "name": "example_fn-0bf17e38-04b4-410c-ae22-e35179eb34ea",
    "context": {
        "trace_id": "0x62cd1ca61b98012a18f99ed325613669",
        "span_id": "0xba365ecb80b88383",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-07-07T12:39:26.164184Z",
    "end_time": "2025-07-07T12:39:26.165149Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {},
    "events": [
        {
            "name": "BaseEvent",
            "timestamp": "2025-07-07T12:39:26.165097Z",
            "attributes": {
                "id_": "762b691e-f96d-41bd-8702-024f057fb3bd",
                "span_id": "example_fn-0bf17e38-04b4-410c-ae22-e35179eb34ea",
                "data": "Hello world!",
                "class_name": "BaseEvent"
            }
        },
        {
            "name": "BaseEvent",
            "timestamp": "2025-07-07T12:39:26.165124Z",
            "attributes": {
                "id_": "c47cbda1-91f2-488

As you can see, we have both our events, which we can recognize by the presence of their `data` and `print_statement` attributes

## 4. Instrument a workflow

Now that we know:
1. How to dispatch span and events
2. How to register those events as OpenTelemetry traces

It's time to use this knowledge to build and instrument a workflow!

The workflow that we want to build is very simple, and involves using OpenAI to analyze some short novels and breaking them down in different parts such as introduction, development of the plot and conclusion.

### 4.1 Define custom events

The first thing that we need to do is to define the custom event that we will us throughout our framework.

We can do it by subclassing the general `Event` classes that the `llama-index-workflows` package provides us with

In [2]:
from workflows.events import StartEvent, Event, StopEvent


class InputTextEvent(StartEvent):
    input_text: str


class AnalyzedTextEvent(StopEvent):
    introduction: str
    development: str
    conclusion: str


class ProgressEvent(Event):
    msg: str

### 4.2 Define custom resources

[Resources](https://docs.llamaindex.ai/en/stable/understanding/workflows/resources/) are a way of performing dependency injection in workflow steps.

We will need just one resource, i.e. an LLM able to produce a structured output that aligns with the text analysis we want to perform.

We need to specify a schema for the structured output first:

In [3]:
from pydantic import BaseModel, Field


class TextAnalysis(BaseModel):
    introduction: str = Field(description="Introduction of the novel")
    development: str = Field(description="Development of the novel")
    conclusion: str = Field(description="Conclusion of the novel")

Let's then initialize an OpenAI LLM as a structured LLM:

In [4]:
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

··········


In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4.1")

In [6]:
struct_llm = llm.as_structured_llm(TextAnalysis)

Let's now define the function that would get us our LLM as resource:

In [7]:
from llama_index.core.llms.structured_llm import StructuredLLM


async def get_llm(*args, **kwargs) -> StructuredLLM:
    return struct_llm

### 4.3 Create the workflow

Finally, after defining events and resources, we can create our workflow:

In [8]:
from workflows import Workflow, step, Context
from workflows.resource import Resource
from llama_index.core.llms import ChatMessage
from typing import Annotated


class TextAnalysisWorkflow(Workflow):
    @step
    async def analyze_text(
        self,
        event: InputTextEvent,
        ctx: Context,
        llm: Annotated[StructuredLLM, Resource(get_llm)],
    ) -> AnalyzedTextEvent:
        response = await llm.achat(
            messages=[
                ChatMessage(
                    role="user",
                    content=f"Analyze the following text: {event.input_text}",
                )
            ]
        )
        ctx.write_event_to_stream(ProgressEvent(msg="Text analyzed successfully"))
        response_json = json.loads(response.message.content)
        return AnalyzedTextEvent(**response_json)

Now we will run the workflow as-is, an you will already see that it produces OpenTelemtry traces.

In [9]:
# first let's get some data
! curl https://raw.githubusercontent.com/run-llama/workflows-observability-support-data/main/data/short-stories/short_story.txt > short_story.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1297  100  1297    0     0   7562      0 --:--:-- --:--:-- --:--:--  7540


In [10]:
# Let's read these data

with open("short_story.txt", "r") as f:
    text = f.read()
text

'Clara wandered through the old town library, seeking quiet more than books. On a dusty shelf near the back, she pulled down a forgotten novel, its spine cracked and pages yellowed. As she flipped it open, a folded letter slipped out and fluttered to the floor. Curious, she picked it up and read the faded ink: a heartfelt message from a soldier named James to someone named Eleanor, dated 1943. He spoke of love, hope, and his promise to return from war.\n\nUnable to shake the letter from her mind, Clara began digging into the town’s history. She scoured archives, interviewed elderly residents, and traced records through the war memorials. Piece by piece, the story came together: James had never made it home. Eleanor had waited, never knowing why he stopped writing. After weeks of searching, Clara finally found her—Eleanor, now 98, living in a quiet nursing home just outside town.\n\nWhen Clara placed the letter in Eleanor’s hands, the old woman wept. Her voice trembled as she read James

We will now use a different instrumentation object, but be careful: you might need to restart the notebook session and re-run all cells apart from the one where we instantiate and start the `instrumentor` object to make it work.

In [11]:
# let's export the spans to a different files, and then start the instrumentation
se_1 = FileSpanExporter(file_path="workflow_1.json")

instrumentor_1 = LlamaIndexOpenTelemetry(
    span_exporter=se_1,
    service_name_or_resource="tracing.a.workflow.1",
)

instrumentor_1.start_registering()

In [12]:
wf = TextAnalysisWorkflow(timeout=800)

handler = wf.run(start_event=InputTextEvent(input_text=text))
async for ev in handler.stream_events():
    if isinstance(ev, ProgressEvent):
        print(ev.msg, flush=True)

result = await handler

Text analyzed successfully


In [13]:
print("Introduction\n\n", result.introduction)
print("\n--\n")
print("Development\n\n", result.development)
print("\n--\n")
print("Conclusion\n\n", result.conclusion)

Introduction

 Clara wandered through the old town library, seeking quiet more than books. On a dusty shelf near the back, she pulled down a forgotten novel, its spine cracked and pages yellowed. As she flipped it open, a folded letter slipped out and fluttered to the floor. Curious, she picked it up and read the faded ink: a heartfelt message from a soldier named James to someone named Eleanor, dated 1943. He spoke of love, hope, and his promise to return from war.

--

Development

 Unable to shake the letter from her mind, Clara began digging into the town’s history. She scoured archives, interviewed elderly residents, and traced records through the war memorials. Piece by piece, the story came together: James had never made it home. Eleanor had waited, never knowing why he stopped writing. After weeks of searching, Clara finally found her—Eleanor, now 98, living in a quiet nursing home just outside town.

--

Conclusion

 When Clara placed the letter in Eleanor’s hands, the old wom

As you can see from the output of the cell where we executed the workflow, our OpenTelemtry instrumentation has wrote several spans to the traces file. We can confirm by printing some of the records out:

In [14]:
with open("workflow_1.json", "r") as f:
    lines = f.readlines()
    for line in lines[-5:]:
        print(json.dumps(json.loads(line), indent=4))

{
    "name": "OpenAI.astructured_predict-4832b122-fc75-47bf-a6f0-c4b8fe609fe9",
    "context": {
        "trace_id": "0xead97a6267821eead6f3e36efd510cc5",
        "span_id": "0xbef1f9081a428b3e",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x5867130dc1f850d1",
    "start_time": "2025-07-07T12:45:54.794486Z",
    "end_time": "2025-07-07T12:45:58.745575Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {},
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "service.name": "tracing.a.workflow.1"
        },
        "schema_url": ""
    }
}
{
    "name": "StructuredLLM.achat-c86a3a30-447a-4ad1-9131-7641ec815908",
    "context": {
        "trace_id": "0xead97a6267821eead6f3e36efd510cc5",
        "span_id": "0x5867130dc1f850d1",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x1f9377ce2c5e5230",
    "start_time": "2025-07-07T12:45:54.786104Z",
    "end_

You could also create a workflow that has customized events, in our case that would be:

In [20]:
# define base events
class TextAnalyzedWorkflowEvent(BaseEvent):
    success: bool


class InputTextWorkflowEvent(BaseEvent):
    input_txt: str

In [21]:
# create an instrumented workflow
class TextAnalysisWorkflow(Workflow):
    @step
    async def analyze_text(
        self,
        event: InputTextEvent,
        ctx: Context,
        llm: Annotated[StructuredLLM, Resource(get_llm)],
    ) -> AnalyzedTextEvent:
        dispatcher.event(InputTextWorkflowEvent(input_txt=event.input_text))
        response = await llm.achat(
            messages=[
                ChatMessage(
                    role="user",
                    content=f"Analyze the following text: {event.input_text}",
                )
            ]
        )
        ctx.write_event_to_stream(ProgressEvent(msg="Text analyzed successfully"))
        dispatcher.event(TextAnalyzedWorkflowEvent(success=True))
        response_json = json.loads(response.message.content)
        return AnalyzedTextEvent(**response_json)

As you can see, in this case we do not put the `@dispatcher.span` decorator before the function, since (as we could see with the example above) every step is already instrumented with a dispatcher!

In [22]:
# Let's re-run the custom instrumented workflow

wf = TextAnalysisWorkflow(timeout=800)

handler = wf.run(start_event=InputTextEvent(input_text=text))
async for ev in handler.stream_events():
    if isinstance(ev, ProgressEvent):
        print(ev.msg, flush=True)

result = await handler

Writing 1 spans to workflow_1.json
Text analyzed successfully


If we now print workflow_1.json, we will see that the tracer has registered also our custom spans:

In [23]:
with open("workflow_1.json", "r") as f:
    lines = f.readlines()
    print(json.dumps(json.loads(lines[-3]), indent=4))

{
    "name": "TextAnalysisWorkflow.analyze_text-cc255572-a7e4-4206-b4b4-00ca43d2fd2f",
    "context": {
        "trace_id": "0x320aed58837e56db846d4db6bd4f9fdb",
        "span_id": "0x3c341ffbd43ab067",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x80995e414cd6ad4e",
    "start_time": "2025-07-07T12:48:33.017811Z",
    "end_time": "2025-07-07T12:48:35.417729Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {},
    "events": [
        {
            "name": "BaseEvent",
            "timestamp": "2025-07-07T12:48:35.417710Z",
            "attributes": {
                "id_": "d3e10d9d-0cb9-4bf1-923f-91f3677ed27c",
                "span_id": "TextAnalysisWorkflow.analyze_text-cc255572-a7e4-4206-b4b4-00ca43d2fd2f",
                "input_txt": "Clara wandered through the old town library, seeking quiet more than books. On a dusty shelf near the back, she pulled down a forgotten novel, its spine cracked and pages yellowed. As

As you can see, these two `BaseEvent` instances are exactly our custom events (we can recognize them by the attributes, `input_txt` and `success`)

This is all for Part 1, in Part 2 we will be diving deeper into a more complex workflow, with many more events and more room for customization... See you there!